In [30]:
from torchtext import data
import torch
import torch.nn as nn
import torch.optim as optim
import random
import datetime
from math import floor

seed = 1
torch.manual_seed(seed)
device = torch.device('cuda')
iterator_device = 0
##uncomment for cpu
#device = torch.device('cpu')
#iterator_device = -1

In [31]:
blurb_field = data.Field(sequential=True, use_vocab=True, lower=True, tokenize="spacy", include_lengths=True)
state_field = data.LabelField(sequential=False, use_vocab=False, tensor_type=torch.FloatTensor,
                              preprocessing=lambda x:1 if x=='successful' else 0)
dataset = data.TabularDataset(path='df_text_eng.csv',format='csv',skip_header=True,fields=[('Unnamed: 0', None),('blurb', blurb_field),('state', state_field)])

In [32]:
train, test, validation = dataset.split(random_state=random.seed(seed), split_ratio=[70,15,15])
print("Training Set Size: ", len(train))
print("Test Set Size: ", len(test))
print("Validation Set Size: ", len(validation))

Training Set Size:  150859
Test Set Size:  32327
Validation Set Size:  32327


In [33]:
#words that appear less than 3 times (2 or less) will be considered unknown words with tag "<unk>", 
#they will have the same word embedding
#vocabulary of the training set will only be used to emulate real world situtaions when the test set is unknown
blurb_field.build_vocab(train,min_freq=3)
print("Vocabulary size used: ",len(blurb_field.vocab))

Vocabulary size used:  28760


In [34]:
batch_size = 64

train_iter = data.BucketIterator(dataset=train, batch_size=batch_size, sort_key=lambda x:len(x.blurb),device=iterator_device,
                                repeat=False, train=True, sort_within_batch=True)
test_iter = data.BucketIterator(dataset=test, batch_size=batch_size, sort_key=lambda x:len(x.blurb),device=iterator_device,
                                train=False, sort_within_batch=True)
validation_iter = data.BucketIterator(dataset=validation, batch_size=batch_size, sort_key=lambda x:len(x.blurb),device=iterator_device,
                                      train=False, sort_within_batch=True)

In [35]:
class GRUNet(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size, padding_idx=None):
        super(GRUNet, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=padding_idx)    # word embedding
        self.gru = nn.GRU(embed_size, hidden_size)  #GRU layer, default activation function is tanh
        self.out = nn.Linear(hidden_size, output_size)   # output layer (Fully Connected)

    def forward(self, x, x_lengths):
        #x -> [sentence length(max, smaller sentences are padded), minibatch size]
        embeds = self.embedding(x)
        
        #embeds -> [sentence length, minibatch size, embedding size]
        embeds = nn.utils.rnn.pack_padded_sequence(embeds,x_lengths)
        all_hiddens, last_hidden = self.gru(embeds)      # activation function for hidden layer
        #the first dimension of the hidden layer is 1 (we use one hidden layer uniderectional GRUs), squeeze removes it to be 
        #able to enter the fully connected layer with dimensions [batch size, hidden size]
        score = self.out(last_hidden.squeeze(0))           
        return score

In [36]:
def calc_accuracy(predictions, y):
    predictions = torch.round(torch.sigmoid(predictions))
    correct = (sum(predictions == y)).float()
    accuracy = correct/len(predictions)
    return accuracy

In [37]:
padding_idx = blurb_field.vocab.stoi['<pad>']
embed_size = 100
hidden_size = 100

In [38]:
grumodel = GRUNet(len(blurb_field.vocab),embed_size,hidden_size,1, padding_idx=padding_idx)
optimizer = optim.SGD(grumodel.parameters(), lr=1e-3)
loss_fn = nn.BCEWithLogitsLoss()
grumodel = grumodel.to(device)
loss_fn = loss_fn.to(device)

In [39]:
def train(model, iterator, optimizer, loss_fn):
    epoch_loss = 0
    epoch_accuracy = 0
    
    model.train()
    
    for minibatch in iterator:
        optimizer.zero_grad()
        predictions = model(minibatch.blurb[0],minibatch.blurb[1]).squeeze(1)
        loss = loss_fn(predictions, minibatch.state)
        accuracy = calc_accuracy(predictions, minibatch.state)    
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_accuracy += accuracy.item()
        
    return epoch_loss / len(iterator), epoch_accuracy / len(iterator)

In [40]:
def evaluate(model, iterator, loss_fn):
    epoch_loss = 0
    epoch_accuracy = 0
    
    model.eval()
    
    with torch.no_grad():
        for minibatch in iterator:
            predictions = model(minibatch.blurb[0],minibatch.blurb[1]).squeeze(1)
            loss = loss_fn(predictions, minibatch.state)
            accuracy = calc_accuracy(predictions, minibatch.state)    

            epoch_loss += loss.item()
            epoch_accuracy += accuracy.item()
        
    return epoch_loss / len(iterator), epoch_accuracy / len(iterator)

In [41]:
no_epochs = 100
#import ipdb; ipdb.set_trace() # debugging starts here
train_accuracies = []
validation_accuracies = []
init_time = datetime.datetime.now()
for epoch in range(no_epochs):
    train_loss, train_accuracy = train(grumodel, train_iter, optimizer, loss_fn)
    validation_loss, validation_accuracy = evaluate(grumodel, validation_iter, loss_fn)
    train_accuracies.append(train_accuracy)
    validation_accuracies.append(validation_accuracy)
    current_time = datetime.datetime.now()
    total_time = (current_time-init_time).total_seconds()
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy*100:.2f}%, ' +  
          f'Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_accuracy*100:.2f}%')
    print(f'Total Time Passed: {floor(total_time/3600)} hours, {floor(total_time/60)%60} minutes, {total_time%60:.2f} seconds')

--Return--
None
> <ipython-input-41-7a88bce7c709>(2)<module>()
      1 no_epochs = 100
----> 2 import ipdb; ipdb.set_trace() # debugging starts here
      3 train_accuracies = []

ipdb> 
ipdb> tbreak 7
Breakpoint 4 at <ipython-input-41-7a88bce7c709>:7
ipdb> c
Deleted breakpoint 4 at <ipython-input-41-7a88bce7c709>:7
None
> <ipython-input-41-7a88bce7c709>(7)<module>()
      6 for epoch in range(no_epochs):
----> 7     train_loss, train_accuracy = train(grumodel, train_iter, optimizer, loss_fn)
      8     validation_loss, validation_accuracy = evaluate(grumodel, validation_iter, loss_fn)

ipdb> s
--Call--
> <ipython-input-39-4bf70a398ede>(1)train()
----> 1 def train(model, iterator, optimizer, loss_fn):
      2     epoch_loss = 0
      3     epoch_accuracy = 0

ipdb> tbreak 4
*** Blank or comment
ipdb> tbreak 3
Breakpoint 5 at <ipython-input-39-4bf70a398ede>:3
ipdb> c
Deleted breakpoint 5 at <ipython-input-39-4bf70a398ede>:3
> <ipython-input-39-4bf70a398ede>(3)train()
      2     epoch_

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torchtext/data/field.py:321: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train), lengths
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


Epoch: 01, Train Loss: 0.6929, Train Accuracy: 51.39%, Validation Loss: 0.6923, Validation Accuracy: 51.62%
Total Time Passed: 0 hours, 0 minutes, 52.58 seconds
Epoch: 02, Train Loss: 0.6919, Train Accuracy: 52.02%, Validation Loss: 0.6916, Validation Accuracy: 51.99%
Total Time Passed: 0 hours, 1 minutes, 15.65 seconds
Epoch: 03, Train Loss: 0.6913, Train Accuracy: 52.46%, Validation Loss: 0.6911, Validation Accuracy: 52.58%
Total Time Passed: 0 hours, 1 minutes, 38.90 seconds
Epoch: 04, Train Loss: 0.6908, Train Accuracy: 52.78%, Validation Loss: 0.6906, Validation Accuracy: 52.59%
Total Time Passed: 0 hours, 2 minutes, 1.94 seconds
Epoch: 05, Train Loss: 0.6904, Train Accuracy: 53.02%, Validation Loss: 0.6902, Validation Accuracy: 52.83%
Total Time Passed: 0 hours, 2 minutes, 25.21 seconds
Epoch: 06, Train Loss: 0.6900, Train Accuracy: 53.21%, Validation Loss: 0.6899, Validation Accuracy: 53.06%
Total Time Passed: 0 hours, 2 minutes, 48.24 seconds

Program interrupted. (Use 'cont' t

BdbQuit: 

In [ ]:
test_loss, test_accuracy = evaluate(grumodel, test_iter, loss_fn)

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy*100:.2f}%')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1,no_epochs+1),train_accuracies)
plt.show()

In [ ]:
with open('GRUModelTraining.txt', 'w') as f:
    for item in train_accuracies:
        f.write("%s\n" % item)

In [ ]:
with open('GRUModelVal.txt', 'w') as f:
    for item in validation_accuracies:
        f.write("%s\n" % item)